In [1]:
# LSTM_Model_Comparison.ipynb
# Vergleich mehrerer LSTM-Modelle für die Vorhersage von Druckluftverbrauch

# Ziel:
# Dieses Notebook vergleicht verschiedene LSTM-Konfigurationen auf Basis desselben Datensatzes.
# Modelle unterscheiden sich in Layer-Anzahl, Dropout, Feature-Auswahl, etc.
# Ergebnisse (Modelle, Plots, Logs) werden in individuellen Ordnern gespeichert.

# --- Imports ---

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.losses import Huber
# Standardbibliotheken
import os
import json
from datetime import datetime

import tensorflow as tf


import sys
import os

from datetime import datetime
DATA_FOLDER = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', 'Data'))
INPUT_FILE = os.path.join(DATA_FOLDER, 'multivariate', 'dataset2_m_v1.csv')



# -------------------- Hilfsfunktionen --------------------
# Verwende os.getcwd(), um den aktuellen Arbeitsordner zu ermitteln
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
# from Machine_Learning_Algorithms.shared_2.read_write import load_train_data_with_datetime, load_test_data_with_datetime, load_test_data_by_fraction, load_train_data_by_fraction
# from Machine_Learning_Algorithms.shared_2.environment import OUTPUT_DATA_PATH_MULTIVARIATE



# Visualisierung
%matplotlib inline
sns.set(style="whitegrid")

In [10]:
#lsese csv aus
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))


import os
# Definiere den Pfad zur CSV-Datei
# file path: C:\Users\ericg\Documents\Mechatronik M Sc\6. Semster\MA\Dev_Ma\Repo\Data\multivariate\Error Metrics
file_path = os.path.join(
    r'C:\Users\ericg\Documents\Mechatronik M Sc\6. Semster\MA\Dev_Ma\Repo_ neu\CompressedAirForecast\Machine_Learning_Algorithms\Edge_Device_sorted\Output\Error_Metrics_1', 
    #'Error Metrics',  # Erster Ordner
    #'Error_Metrics_1',  # Zweiter Ordner (fehlte im Original)
    'metrics_summary.csv'  # Datei
)
def read_csv_file(file_path):
    try:
        df = pd.read_csv(file_path, sep=',', decimal=',')
        return df
    except Exception as e:
        print(f"Fehler beim Lesen der CSV-Datei: {e}")
        return None
df = read_csv_file(file_path)

In [11]:
df

,paths_base,paths_input,paths_output,paths_input_data_edge_device,paths_output_data_edge_device,paths_input_data_server,paths_output_data_server,model_name,dataset,model_type,...,mape,smape,wape,msle,median_ae,mase,n_estimators,max_depth,random_state,n_jobs
0,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,linreg_test,dataset2_m_v1.csv,ridge,...,9.81100174303355e-09,9.811001740764432e-09,3.699371995276808e-09,4.63772610470152e-20,5.47814238416322e-09,1.1704367259883884e-10,NaN,NaN,NaN,NaN
1,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,linreg_test,dataset2_m_v1.csv,ridge,...,9.81100174303355e-09,9.811001740764432e-09,3.6993719952768082e-09,4.63772610470152e-20,5.47814238416322e-09,1.1704367259883884e-10,NaN,NaN,NaN,NaN
2,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,C:\Users\ericg\Documents\Mechatronik M Sc\6. S...,rf_test,dataset2_m_v1.csv,NaN,...,8.002849175294953,7.613632025389176,5.731352117699729,0.012609221437736837,12.563587739883872,0.18133307536229926,5.0,3.0,42.0,1.0


In [15]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
print(pio.renderers.default) 

import importlib.metadata

try:
    nbformat_version = importlib.metadata.version("nbformat")
    print(f"Installierte nbformat Version: {nbformat_version}")
except importlib.metadata.PackageNotFoundError:
    print("nbformat ist nicht installiert.")

def visualize_predictions_interactive(file_path, output_dir=None):
    """
    Interaktive Visualisierung von Vorhersagen mit Plotly – mit synchronem Zoom über die Zeitachse.
    Erwartet eine CSV mit Spalten: 'date', 'true_h*', 'pred_h*'
    """
    try:
        # CSV einlesen
        df = pd.read_csv(file_path)

        # 'date' als Zeitstempel parsen (enthält bereits Datum + Uhrzeit mit ms)
        df['datetime'] = pd.to_datetime(df['date'])

        # Relevante Spalten finden
        true_cols = [col for col in df.columns if col.startswith('true_h')]
        pred_cols = [col for col in df.columns if col.startswith('pred_h')]

        num_plots = len(true_cols)
        fig = make_subplots(
            rows=num_plots, cols=1,
            shared_xaxes=True,
            subplot_titles=[f'Horizon {i+1}' for i in range(num_plots)],
            vertical_spacing=0.05
        )

        for i, (true_col, pred_col) in enumerate(zip(true_cols, pred_cols)):
            row = i + 1

            # Echtdaten
            fig.add_trace(go.Scatter(
                x=df['datetime'], y=df[true_col],
                mode='lines', name='Echt',
                line=dict(color='blue'),
                showlegend=(i == 0)
            ), row=row, col=1)

            # Vorhersage
            fig.add_trace(go.Scatter(
                x=df['datetime'], y=df[pred_col],
                mode='lines', name='Vorhersage',
                line=dict(dash='dash', color='red'),
                showlegend=(i == 0)
            ), row=row, col=1)

        fig.update_layout(
            height=300 * num_plots,
            width=1200,
            title_text="Interaktive Vorhersagevisualisierung (synchroner Zoom)",
            title_font=dict(size=22),
            margin=dict(t=80, b=50),
            legend=dict(x=1.01, y=1, font=dict(size=14)),
        )

        fig.update_xaxes(title_text="Zeit", tickangle=0, tickfont=dict(size=12))
        fig.update_yaxes(title_text="Wert", tickfont=dict(size=12))

        fig.show()

    except Exception as e:
        print(f"❌ Fehler beim interaktiven Visualisieren: {str(e)}")
        raise

vscode
nbformat ist nicht installiert.


In [16]:
# Beispielaufruf
file_path = r"C:\Users\ericg\Documents\Mechatronik M Sc\6. Semster\MA\Dev_Ma\Repo_ neu\CompressedAirForecast\Machine_Learning_Algorithms\Edge_Device_sorted\Output\Prediction Data\predictions_2025-06-23_124543_1767_2025-06-23_124543.csv"
visualize_predictions_interactive(file_path)

❌ Fehler beim interaktiven Visualisieren: Mime type rendering requires nbformat>=4.2.0 but it is not installed


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed